# Frequency Shifting

Explore frequency shifting via _single sideband modulation_ (SSB).
We start at the effects of _amplitude modulation_ and introduce the _Hilbert transform_ and the _analytical signal_ to develop SSB.

For more information see **DAFX - Zölzer (Ch. Modulators and demodulators)** and Wikipedia.


## Amplitude Modulation

A signal $x$ is multiplied by a harmonic function:

$$x_{AM} =  (A_c + A_m \cdot x) \cdot cos(\Omega t)$$

In [ ]:
# load needed modules
import numpy as np
from ipywidgets import *
import IPython.display as ipd
from scipy.io import wavfile
import scipy.signal as signal
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 3]
from scipy.fft import rfft, fft, rfftfreq, fftfreq
from scipy.signal import hilbert, chirp

# load an audio signal
fs, x = wavfile.read("../../samples/Toms_diner.wav")
x = x / np.abs(x.max())

plt.plot(x)
plt.title("original")
ipd.Audio(x, rate=fs)

In [ ]:
# generate the carrier signal
t = np.linspace(0, x.size/fs, x.size)
f = 80 # Hz
w = 2 * np.pi * f
carrier = np.cos(w * t)

plt.plot(carrier)
plt.title("carrier")

In [ ]:
# do the amplitude modulation
A_c = 1
A_m = 0.5
x_am = (A_c + A_m * x) * carrier

plt.plot(x_am)
plt.title("modulated signal")
ipd.Audio(x_am, rate=fs)

Let's consider, what the AM does to the spectrum of a signal.
For a single sinusoid as input signal $x$, the resulting spectrum is a carrier bin and an upper and a lower side band.
These are spaced around the carrier bin by the frequency $\omega$.

In [ ]:
# let's define a function for plotting the magnitude spectrum
def plot_mag_spec(sig, fs, name="", negative=False, f_range=(20, 20000), db_range=(-40, 0), f_log=True):
    """
    Plot the magnitude spectrum of the signal 'sig'.
    sig:    signal to be analyzed
    fs:     sampling rate of sig
    name:   name of the plotted spectrum
    negative: show negative frequencies (default: False)
    f_range: define a frequency range for the spectrum (default: 20Hz to 20kHz)
    db_range: define a magnitude range in dB for the spectrum (default: 0dB to 100dB)
    f_log:  logarithmic or linear frequency axis? (default: True is logarithmic)
    """
    w = signal.hann(sig.size) # window
    if negative == False:
        # just positive frequencies
        freq = rfftfreq(sig.size, 1 / fs)
        mag = 20*np.log10(np.abs((1/sig.size)*rfft(sig*w)))
        if f_log == True:
            plt.semilogx(freq, mag)
        else:
            plt.plot(freq, mag)
        plt.xlim(f_range)
    else:
        # positive and negative frequencies
        freq = fftfreq(sig.size, 1 / fs)
        mag = 20*np.log10(np.abs((1/sig.size)*fft(sig*w)))
        if f_log == True:
            freq = freq[1:] # omit 0Hz
            mag = mag[1:]
        plt.plot(freq, mag)
        plt.xlim((-f_range[1], f_range[1]))
        if f_log == True:
            plt.xscale('symlog')
    plt.ylim(db_range)
    plt.xlabel("f [Hz]")
    plt.ylabel("amplitude [dB]")
    plt.title(name)
    plt.show()

In [ ]:
# interactive AM spectrum
def AM(f_carrier=5000, f=1000):
    # generate signals
    fs = 44100
    dur = 0.02
    t = np.linspace(0, dur, round(dur*fs))
    carrier = np.cos(2*np.pi*f_carrier*t)
    x = np.cos(2*np.pi*f*t)
    a_carrier = 1
    a = 0.25
    x_am = (a_carrier + a * x) * carrier
    plot_mag_spec(x_am, fs, "AM spectrum", negative=True, f_log=False)

interact(AM);

This simple case shows, what happens to every frequency in the base band spectrum, which is an audio signal for instance.
What happens is, that the spectrum of every real signal is symmetrical around $f = 0$.
The amplitude modulation shifts this symmetric spectrum to the carrier frequency
($\Omega + \omega$ and $\Omega - \omega$).

Let's see, what happens to bandlimited noise as base band signal:

In [ ]:
sig_len = 20*1024
fs = 44100
x = np.random.normal(0, 1, sig_len)

# lowpass filter
fc = 1000
sos = signal.butter(6, fc, btype='lowpass', analog=False, output='sos', fs=fs)
x = signal.sosfilt(sos, x)

plot_mag_spec(x, fs, name="bandlimited noise spectrum", db_range=(-50, -30), f_log=False)

In [ ]:
f_carrier = 5000
t = np.linspace(0, sig_len/fs, sig_len)
carrier = np.cos(2*np.pi*f_carrier*t)
A_c = 1
A_m = 1
x_am = (A_c + A_m * x) * carrier

plot_mag_spec(x_am, fs, name="AM with baseband spectrum", db_range=(-60, -10), f_log=False)

So, the whole symmetric spectrum gets shifted up by the carrier frequency.


## The Hilbert Transform and the Analytical Signal

For defining the single sideband modulation, we first need the hilbert transform and the concept of the analytical signal.


### Hilbert Transform

The Hilbert transform is given by the convolution (from [here](https://de.wikipedia.org/wiki/Hilbert-Transformation))

$$g(t) = f(t) * \frac{1}{\pi t}.$$

The Fourier transform of the continuous Hilbert transform is given by

$$H_H(j\omega) = -j \cdot sgn(\omega)$$

This corresponds to a phase shift of $\pi/2$ or $+90^°$ for negative frequencies and of $-\pi/2$ or $-90^°$ for positive frequencies.

The discrete Hilbert transform can be done by the non causal FIR filter given by the impulse response:

$$
h[n] = \frac{1 - cos(\pi n)}{\pi n} = \begin{cases} 2/\pi n & for\: n \: odd \\ 0 & for\: n \: even \end{cases}
$$

To make the filter causal, $h[n]$ is truncated to a finite length $N$ and time shifted to causality.
Because of that, the FIR Hilbert filter requires a compensation delay of the other signals, which do not use this filter.
Note, that the discrete Hilbert transform is not the sampled continuous Hilbert transform.


### Analytic Signal

The analytic signal $x_a[n]$ is a complex signal whose imaginary part is the Hilbert transform ($H(\cdot)$) of the real part according to the real signal $x[n]$:

$$x_a[n] = x[n] + jH(x[n])$$

The analytic signal has a **single sided spectrum** which is not symmetric like spectra of real signals.

With the analytic signal $x_a[n]$, we can calculate

* the *amplitude envelope* of $x[n]$

* the *instantaneous phase* of $x[n]$


### Implementing the Analytical Signal

Scipy provides a function [scipy.signal.hilbert()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.hilbert.html#r37d8c8a6fd16-1) for generating the analytical signal $x_a[n]$ to the real signal $x[n]$.
Let's determine the amplitude envelope and instantaneous frequency of an amplitude-modulated signal (taken from the link above).

In [ ]:
dur = 1
fs = 44100
samples = int(fs*dur)
t = np.arange(samples) / fs

# chirp from 20Hz to 100Hz
x = chirp(t, 20.0, t[-1], 100.0)

# apply amplitude modulation
x *= (1.0 + 0.5 * np.sin(2.0*np.pi*3.0*t) )

* "The amplitude envelope is given by magnitude of the analytic signal.
* The instantaneous frequency can be obtained by differentiating the instantaneous phase in respect to time.
* The instantaneous phase corresponds to the phase angle of the analytic signal."

In [ ]:
x_a = hilbert(x)

amplitude_envelope = np.abs(x_a)
instantaneous_phase = np.unwrap(np.angle(x_a))
instantaneous_frequency = np.diff(instantaneous_phase) / (2.0*np.pi) * fs

# plot
fig = plt.figure()
ax0 = fig.add_subplot(211)
ax0.plot(t, x, label='x[n]')
ax0.plot(t, amplitude_envelope, label='envelope')
ax0.set_xlabel("time in seconds")
ax0.legend()
ax1 = fig.add_subplot(212)
ax1.plot(t[1:], instantaneous_frequency)
ax1.set_xlabel("time in seconds")
ax1.set_ylim(0.0, 120.0)

## Single Sideband Modulation (SSB)

(from [wiki](https://en.wikipedia.org/wiki/Single-sideband_modulation))

The AM spectrum consists of the carrier signal and the two symmetric side bands.
This is a lot of unneeded information, which uses up bandwidth.
All the information is contained in one side band.

The analytical signal is used to just get the upper or the lower side band of a spectrum.
Given are the carrier signal $x[n]$ and the modulator signal $m[n]$.
(For real-time implementations, the compensation delays according to the Hilbert filter have to be applied to the untransformed signals.)

Let's do SSB with a base band signal consisting of three harmonically spaced sinusoids:

In [ ]:
# generate base band signal
fs = 44100
sig_len = 20*1024
t = np.linspace(0, sig_len/fs, sig_len)

s1 = np.cos(2*np.pi*1000*t)
s2 = 0.3 * np.cos(2*np.pi*2000*t)
s3 = 0.1 * np.cos(2*np.pi*3000*t)
m = s1 + s2 + s3

plot_mag_spec(m, fs, "modulator base band spectrum", negative=True, f_log=False)

Now we generate the analytic signal $m_a[n] = m[n] + jH(m[n])$

In [ ]:
m_a = hilbert(m)
plot_mag_spec(m_a, fs, "analytic signal spectrum", negative=True, f_log=False)

The spectrum of the analytical signal of the modulator base band signal is single sided.
The modulation for shifting the base band signal to a higher frequency band takes a complex harmonic function:

$$m_{SSBa} = m_{SSB} + jH(m_{SSB}) = \mathcal{F}^{-1}\{M_a(f-f_0)\} = m_a(t) \cdot e^{j 2 \pi f_0 t}$$

$$m_{SSBa} = m_a(t) \cdot (cos(2 \pi f_0 t) + j\: sin(2 \pi f_0 t))$$

In [ ]:
# modulation
f0 = 10000
x = np.exp(1j*2*np.pi*f0*t)
m_ssba = m_a * x

plot_mag_spec(m_ssba, fs, "modulated analytic signal spectrum", negative=True, f_log=False)

Since a complex signal cannot be sent over a single transmission medium, we just send the real part of the modulated single sideband signal.
The spectrum of the real part is again symmetric around $f=0$.

In [ ]:
m_ssb = m_ssba.real

plot_mag_spec(m_ssb, fs, "SSB spectrum", negative=True, f_log=False)

## Audio Frequency Shifting

So let's take the communication theory and shift some real audio with SSB.

In [ ]:
# load the original audio signal
fs, m = wavfile.read("../../samples/Toms_diner.wav")
m = m / np.abs(m.max())

plot_mag_spec(m, fs, "original spectrum", db_range=(-100, -35))
ipd.Audio(m, rate=fs)

In [ ]:
# do frequency shifting
f_shift = 100

t = np.linspace(0, m.size/fs, m.size)
x = np.exp(1j*2*np.pi*f_shift*t)

m_shifted = (hilbert(m) * x).real

plot_mag_spec(m_shifted, fs, "frequency shifted spectrum", db_range=(-100, -35))
ipd.Audio(m_shifted, rate=fs)